In [8]:
import dask.distributed
import dask
import dask_image.imread
import skimage.io as io

import numpy as np
from scipy.signal import convolve2d
import os 

from Models.features import lpq, lpq_plus, LPQ, LPQPlus, PHOG_Algorithm

from sklearn.svm import SVC
from sklearn import metrics

In [2]:
# Constants and Vaiables
IMG_TRAIN_PATH = "../Assets/data/train"
IMG_TEST_PATH = "../Assets/data/test"
HAPPY_PATH = "../Assets/data/train/happy/"

train_folders = os.listdir(IMG_TRAIN_PATH)  # Emotions Folders
test_folders = os.listdir(IMG_TEST_PATH)  # Emotions Folders

# Intialize dash board for DASK
# client = dask.distributed.Client()  # Create a local cluster  
cluster = dask.distributed.LocalCluster(n_workers=12, threads_per_worker=2)
client = dask.distributed.Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 12
Total threads: 24,Total memory: 15.91 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:59841,Workers: 12
Dashboard: http://127.0.0.1:8787/status,Total threads: 24
Started: Just now,Total memory: 15.91 GiB
Comm: tcp://127.0.0.1:59936,Total threads: 2
Dashboard: http://127.0.0.1:59939/status,Memory: 1.33 GiB
Nanny: tcp://127.0.0.1:59847,


### **Auxilary Functions:**

In [3]:
# Read Image Function Call
@dask.delayed
def read_img_parallel(img_path):
    return io.imread(img_path, as_gray=True)

# Reading A Directory of images folders
def read_data(path: str):
    x_img = []
    y_img = []
    tasks = []
    folders = os.listdir(path)
    for folder in folders:
        folder_path = os.path.join(path, folder)
        pattern = os.path.join(folder_path, 'im*.png')
        imgs = dask_image.imread.imread(pattern)
        tasks.append(imgs)
        # Set Labels While Initializing
        labels = [folder for _ in range(len(imgs))]
        y_img.extend(labels)
    
    print("Implementing Tasks...")
    for task in tasks:
        x_img.extend(task.compute())
        print("Done...")
    return x_img, y_img

def get_phog(images):
    phog = []
    for img in images:
        phog_desc = dask.delayed(PHOG_Algorithm)(img, 8, 3)
        phog.append(phog_desc)

    print('Starting PHOG Set...')
    phog = dask.compute(phog)[0]
    print('Done PHOG Set...')
    return phog

def get_lpq(images):
    lpq_train = []
    for img in images:
        lpq_desc = dask.delayed(lpq)(img)
        lpq_train.append(lpq_desc)

    print('Starting LPQ Set...')
    lpq_train = dask.compute(lpq_train)[0]
    print('Done LPQ Set...')
    return lpq_train
        

**Different Implementations for the Delayed function Parallelization**  
1- Using multiple dask objects

## Different Methods for Reading the Images:
**1- Using Separete Delayed Functions.**  
_Quiet Fast ~ 1.5 mins_

In [ ]:
# Reading Training Images ~ 1.6 mins
x_img = []
y_train = []
tasks = []
# Create the reading Tasks and add them to a list -> tasks
for folder in train_folders:
    folder_path = os.path.join(IMG_TRAIN_PATH, folder)
    pattern = os.path.join(folder_path, 'im*.png')
    imgs = dask_image.imread.imread(pattern)
    tasks.append(imgs)
    # Set Labels While Initializing
    labels = [folder for _ in range(len(imgs))]
    y_train.extend(labels) 
    
# Distribute Tasks to Cores and Threads 
print("Implementing Tasks...")
for task in tasks:
    x_img.extend(task.compute())
    print("Done...")

In [ ]:
# Reading Test Images ~ 30 sec
x_test_img = []
y_test = []
tasks = []
# Create the reading Tasks and add them to a list -> tasks


# Distribute Tasks to Cores and Threads 
print('implementing Tasks')
for task in tasks:
    x_test_img.extend(task.compute())
    print("Done...")

In [16]:
# Verifying Results
len(y_train), len(x_img), len(x_test_img), len(y_test)

(28709, 28709, 7178, 7178)

**2- Using list of delayed Functions.**  
_Looks Too Slow ~ 5 mins_

In [7]:
# Reading Train Dataset
x_img = []
y_img = []
for folder in train_folders:
    folder_path = os.path.join(IMG_TRAIN_PATH, folder)
    pattern = os.path.join(folder_path, 'im*.png')
    imgs = dask_image.imread.imread(pattern)
    x_img.extend(imgs)
    labels = [folder for _ in range(len(imgs))]
    y_img.extend(labels) 

print("Implementing Tasks")
x_img = dask.compute(x_img)[0]
print('Done...')

In [ ]:
# Reading Test Dataset
x_test_img = []
y_test = []
for folder in test_folders:
    folder_path = os.path.join(IMG_TEST_PATH, folder)
    pattern = os.path.join(folder_path, 'im*.png')
    imgs = dask_image.imread.imread(pattern)
    x_test_img.extend(imgs)
    labels = [folder for _ in range(len(imgs))]
    y_test.extend(labels) 

print("implementing Reads")
x_test_img = dask.compute(x_test_img)[0]
print("Done...")

#### **Applying Feature Extraction for all the images**

#####  1- LPQ and LPQ_PLUS.   

In [ ]:
# Training and Test Images LPQ Feature Extraction
lpq_train = []
for img in x_img:
    lpq_desc = dask.delayed(lpq)(img)
    lpq_train.append(lpq_desc)

print('Starting LPQ Train Set...')
lpq_train = dask.compute(lpq_train)[0]
print('Done LPQ Train Set...')

# Testing Images
lpq_test = []
for img in x_test_img:
    lpq_desc = dask.delayed(lpq)(img)
    lpq_test.append(lpq_desc)

print('Starting LPQ Test Set...')
lpq_test = dask.compute(lpq_test)[0]
print('Done LPQ Test Set...')

### PHOG Feature Extraction -> Very Slow!!!

In [ ]:
# Training and Test Images PHOG Feature Extraction
# Training Images
phog_train = []
for img in x_img:
    phog_desc = dask.delayed(PHOG_Algorithm)(img, 8, 3)
    phog_train.append(phog_desc)

print('Starting...')
phog_train = dask.compute(phog_train)[0]
print('Done...')

# Test Images
phog_test = []
for img in x_test_img:
    phog_desc = dask.delayed(PHOG_Algorithm)(img, 8, 3)
    phog_test.append(phog_desc)

print('Starting...')
phog_test = dask.compute(phog_test)[0]
print('Done...')

In [25]:
phog_train[0].shape, lpq_train[0].shape

((680,), (255,))

In [29]:
training_data = np.concatenate((lpq_train, phog_train), axis=1)
testing_data = np.concatenate((lpq_test, phog_test), axis=1)
training_data.shape, testing_data.shape, (255+680)

((28709, 935), (7178, 935), 935)

# **The pipeline of Emotion Module**

### **1. Reading the dataset (training, test):**

In [4]:
x_img, y_train = read_data(IMG_TRAIN_PATH)
x_img_test, y_test = read_data(IMG_TEST_PATH)

Implementing Tasks...
Done...
Done...
Done...
Done...
Done...
Done...
Done...
Implementing Tasks...
Done...
Done...
Done...
Done...
Done...
Done...
Done...


### **2.  Extracting Features from Images:**

##### 2.1  **Features extraction (preparing x-train, x_test)**

In [6]:
lpq_train_feat = get_lpq(x_img)
lpq_test_feat = get_lpq(x_img_test)

phog_train_feat = get_phog(x_img)
phog_test_feat = get_phog(x_img_test)

Starting LPQ Set...
Done LPQ Set...
Starting LPQ Set...
Done LPQ Set...
Starting PHOG Set...
Done PHOG Set...
Starting PHOG Set...
Done PHOG Set...


##### 2.2 **Building the Features Dataframe.**

In [9]:
x_train = np.concatenate((lpq_train_feat, phog_train_feat), axis=1)
x_test = np.concatenate((lpq_test_feat, phog_test_feat), axis=1)
x_train.shape, x_test.shape, (255+680)

((28709, 935), (7178, 935), 935)

### **3. Model Training and Classification:**

##### 3.1 **Initalizing the Classifier.** acc = 40%

In [9]:
#Create a svm Classifier
clf = SVC()
print("classifier initialized...")

#Train the model using the training sets
clf.fit(x_train, y_train)
print("classifier Trained...")

#Predict the response for test dataset
y_pred = clf.predict(x_test)

classifier initialized...
classifier Trained...


##### 3.2 **Classifier with Data Shuffling.**

In [10]:
from sklearn.utils import shuffle
x_shuffled, y_shuffled = shuffle(x_train, y_img)
x_test_shuffled, y_test_shuffled = shuffle(x_test, y_test)
#Create a svm Classifier
clf = SVC()
print("Classifier initialized...")

#Train the model using the training sets
clf.fit(x_shuffled, y_shuffled)
print("Classifier Trained...")

#Predict the response for test dataset
y_pred_shuffled = clf.predict(x_test_shuffled)

Classifier initialized...
Classifier Trained...


### **4. Evaluating the model**

In [11]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred, average=None))
print("Recall:",metrics.recall_score(y_test, y_pred, average=None))

Accuracy: 0.41195319030370575
Precision: [0.37611408 1.         0.3472     0.45454545 0.34863946 0.3554717
 0.52844501]
Recall: [0.22025052 0.04504505 0.21191406 0.69052988 0.3325223  0.3777065
 0.50300842]


In [12]:
print("Accuracy:",metrics.accuracy_score(y_test_shuffled, y_pred_shuffled))
print("Precision:",metrics.precision_score(y_test_shuffled, y_pred_shuffled, average=None))
print("Recall:",metrics.recall_score(y_test_shuffled, y_pred_shuffled, average=None))

Accuracy: 0.41195319030370575
Precision: [0.37611408 1.         0.34664537 0.45471418 0.34863946 0.3554717
 0.52844501]
Recall: [0.22025052 0.04504505 0.21191406 0.69052988 0.3325223  0.3777065
 0.50300842]


### **5. Parameters Tunning using Grid Search:**

In [10]:
from sklearn.model_selection import GridSearchCV
# SVC(C=1, gamma=1, kernel='poly')
param_grid = {'C': [0.1, 1, 10], 'gamma': [1, 0.05 ,0.001],'kernel': ['poly', 'sigmoid']}
grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=2, n_jobs=-1)
grid.fit(x_train, y_train)
y_pred_grid = grid.predict(x_test)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


In [11]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_grid))
print("Precision:",metrics.precision_score(y_test, y_pred_grid, average=None))
print("Recall:",metrics.recall_score(y_test, y_pred_grid, average=None))

Accuracy: 0.4474784062412928
Precision: [0.3410302  0.47107438 0.36766034 0.55390539 0.40018067 0.36339286
 0.6508344 ]
Recall: [0.40083507 0.51351351 0.39746094 0.56764374 0.35928629 0.32638332
 0.6101083 ]


In [12]:
grid.best_estimator_

SVC(C=1, gamma=1, kernel='poly')

2022-04-25 15:57:18,479 - tornado.application - ERROR - Uncaught exception GET /status/ws (127.0.0.1)
HTTPServerRequest(protocol='http', host='127.0.0.1:8787', method='GET', uri='/status/ws', version='HTTP/1.1', remote_ip='127.0.0.1')
Traceback (most recent call last):
  File "d:\Grad. Project\GP2\Codes\repo_emotion\venv\lib\site-packages\tornado\websocket.py", line 954, in _accept_connection
    open_result = handler.open(*handler.open_args, **handler.open_kwargs)
  File "d:\Grad. Project\GP2\Codes\repo_emotion\venv\lib\site-packages\tornado\web.py", line 3173, in wrapper
    return method(self, *args, **kwargs)
  File "d:\Grad. Project\GP2\Codes\repo_emotion\venv\lib\site-packages\bokeh\server\views\ws.py", line 149, in open
    raise ProtocolError("Token is expired.")
bokeh.protocol.exceptions.ProtocolError: Token is expired.


## END 

In [ ]:
# PipeLine Gray, LPQ(+), PHOG, Class
# Emotic Dataset
# SVM Parameters Tuning